<a href="https://colab.research.google.com/github/olaviinha/WaveNet-colab/blob/master/WaveNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## WaveNet – Train & generate (44.1kHz sample rate)

This is a slightly modified version of [A TensorFlow implementation of DeepMind's WaveNet paper](https://github.com/ibab/tensorflow-wavenet) to be run in Google Colab, using Google Drive as data storage.

<b>Howto</b>
*   Set all direcotries (`***_dir` variables under <i>Settings</i>) relative to your Google Drive root.
*   Select <i>Runtime > Run all</i> from the menu.

<font color="#d99">Current bugs:</font>
*   Due to version prerequisition mismatches in Colab, this is currently unable to continue training from any previous checkpoint. Though it may come as consolation that you can exit Colab and leave it running, easily achieving 1e5 steps in one go.



In [ ]:
#@title Mount Drive & clone repository

%tensorflow_version 1.x
from google.colab import drive
drive.mount('/content/drive')
%cd /content
!git clone https://github.com/olaviinha/tensorflow-wavenet.git
%cd /content/tensorflow-wavenet
!git pull

In [ ]:
#@title ## Settings
#@markdown <pre><b>Common</b></pre>
##@markdown <small>`generate_every` is the number of steps to Train every time before generating audio from achieved checkpoint. This is required only when Train & Generate is selected from the first dropdown.</small><br>
#@markdown <small>`checkpoints_dir` is the directory where checkpoints will be <font color="#d99">saved to when training</font> and <font color="#d99">read from when generating</font>.</small>
what_to_do = "Generate" #@param ["Train", "Generate"]
#generate_every = 5000 #@param {type:"integer"}
checkpoints_dir = "ai/training/wavenet" #@param {type:"string"}

#@markdown <pre><b>Training settings</b> <font color="#888">(ignore when generating)</font></pre>
#@markdown <small>Directory of WAV files to train from:</small>
corpus_dir = "ai/input/audiofiles" #@param {type:"string"}

#@markdown <pre><b>Generation settings</b> <font color="#888">(ignore when training)</font></pre>
#@markdown <small>`seconds` is the duration of generated audio.</small><br>
#@markdown <small>Leave `checkpoint` blank to use latest found checkpoint from `checkpoints_dir`. Otherwise path to checkpoint (relative to `checkpoints_dir`<i>/train/</i>). Example value: `2020-07-15T14-30-32/model.ckpt-32050`</small>
output_dir = "ai/output/wavenet" #@param {type:"string"}
seconds = 10 #@param {type:"slider", min:5, max:60, step:1}
checkpoint = "" #@param {type:"string"}

checkpoint_input = checkpoint;
samplerate = 44100
samples = str(samplerate*seconds)

drive_root = "/content/drive/My Drive/"
checkpoints_dir = drive_root+checkpoints_dir
corpus_dir = drive_root+corpus_dir
output_dir = drive_root+output_dir

import ntpath
import random
import string
from glob import glob
from IPython.display import Audio

def remove_slash(s):
  if s.endswith('/'):
    s = s[:-1]
  return s

def rndStr(length):
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    return result_str

def path_leaf(path):
    head, tail = ntpath.split(path)
    return tail or ntpath.basename(head)

def opts(mode, output):
  if checkpoint_input is "":
    trainDirs = glob(checkpoints_dir+"/train/*/")
    trainDirs.sort()
    last = trainDirs[-1]
    checkpoints = glob(last+"/*")
    checkpoints.sort()
    checkpoint = checkpoints[-1]
  else:
    checkpoint = checkpoints_dir+"/train/"+checkpoint_input

  if mode is "gen":
    checkpoint = checkpoint.replace(".meta", "").replace(".index", "").replace(".data-00000-of-00001", "")
    ckpt = path_leaf(checkpoint).replace("model.ckpt-", "")
    output_file = output_dir+"/generated-ckpt-"+ckpt+"-"+str(seconds)+"s-"+rndStr(4)+".wav"
  else:
    checkpoint = checkpoint.replace(path_leaf(checkpoint), "")
  if output is "output":
    return output_file
  else:
    return checkpoint

remove_slash(checkpoints_dir)
remove_slash(corpus_dir)
remove_slash(output_dir)

In [ ]:
%cd /content/tensorflow-wavenet
if what_to_do is "Generate":
  cp = opts("gen", "cp")
  out = opts("gen", "output")
  print('\n>> Generate from checkpoint', cp)
  !python generate.py --samples={samples} --wav_out_path="{out}" "{cp}"
  print('\n>> Generated file saved to', out)
  Audio(output_file)
elif what_to_do is "Train":
  cp = opts("tra", "cp")
  print('\n>> Restore checkpoint', cp)
  !python train.py --data_dir="{corpus_dir}" --logdir_root="{checkpoints_dir}" --restore_from="{cp}" --silence_threshold=0
else:
  while True:
    cp = opts("tra", "cp")
    print('\n>> Restore checkpoint', cp)
    !python train.py --num_steps={generate_every} --data_dir="{corpus_dir}" --logdir_root="{checkpoints_dir}" --restore_from="{cp}" --silence_threshold=0
    cp = opts("gen", "cp")
    out = opts("gen", "output")
    print('\n>> Generate from checkpoint', cp)
    !python generate.py --samples={samples} --wav_out_path="{out}" "{cp}"
    print('\n>> Generated file saved to', out)
    continue